In [1]:
#!pip install python-dotenv
#!pip install google-generativeai
#!pip install faiss-cpu

In [2]:
# for getting google_api_key from the .env file
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain.llms import GooglePalm

In [4]:
llm=GooglePalm(google_api_key=os.environ["GOOGLE_API_KEY"],temperature=0.1)

### Load data from FAQ csv file

In [5]:
from langchain.document_loaders.csv_loader import CSVLoader

In [6]:
loader=CSVLoader(file_path='codebasics_faqs.csv', source_column="prompt")
doc=loader.load()

In [7]:
type(doc)

list

In [8]:
doc[0]

Document(page_content='prompt: I have never done programming in my life. Can I take this bootcamp?\nresponse: Yes, this is the perfect bootcamp for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in your current job or business using data.', metadata={'source': 'I have never done programming in my life. Can I take this bootcamp?', 'row': 0})

In [9]:
doc[0].page_content

'prompt: I have never done programming in my life. Can I take this bootcamp?\nresponse: Yes, this is the perfect bootcamp for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in your current job or business using data.'

In [10]:
doc[0].metadata

{'source': 'I have never done programming in my life. Can I take this bootcamp?',
 'row': 0}

### Embedding

In [11]:
from langchain.embeddings import GooglePalmEmbeddings

In [12]:
google_embedding=GooglePalmEmbeddings(google_api_key=os.getenv('GOOGLE_API_KEY'))

In [13]:
#Sample embedding
e=google_embedding.embed_query("Sun rises in the east")
print(len(e))
e[:4]

768


[-0.017065058, 0.025053214, -0.041459456, -0.011060075]

## Vector Database using FAISS

In [14]:
from langchain.vectorstores import FAISS

In [15]:
vectorDB=FAISS.from_documents(documents=doc, embedding=google_embedding)

In [16]:
type(vectorDB)

langchain.vectorstores.faiss.FAISS

In [17]:
#Storing it in local drive
vectorDB.save_local("VectorDB_file")

## Load from saved VectorDB and create Retriever from it

In [18]:
# Loading from the saved VectorDB

vectorDB_saved=FAISS.load_local("VectorDB_file",google_embedding)

In [19]:
#Create an Retriever object for querying the vector database

retriever=vectorDB_saved.as_retriever(score_threshold=0.7)

## Prompt Engineering and Creating RetrievalQA chain

In [20]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""

prompt=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
prompt

PromptTemplate(input_variables=['context', 'question'], template='Given the following context and a question, generate an answer based on this context only.\nIn the answer try to provide as much text as possible from "response" section in the source document context without making much changes.\nIf the answer is not found in the context, kindly state "I don\'t know." Don\'t try to make up an answer.\n\nCONTEXT: {context}\n\nQUESTION: {question}')

In [21]:
from langchain.chains import RetrievalQA

chain=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    input_key="query",
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [22]:
# Final response

def response(user_query):
    res=chain(user_query)
    return res['result']

Sample Responses

In [26]:
response('Do you provide job assistance and also do you provide job gurantee?')

'Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters. The courses included in this bootcamp are done by 9000+ learners and many of them have secured a job which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.'

In [27]:
response('Did you provide EMI option?')

'No'

In [28]:
response('Explain about refund policy?')

'As promised we will give you a 100% refund based on the guidelines (Please refer to our course refund policy before enrolling).'

In [29]:
response("Do you have Python courses?")

'Yes, we have Python courses.'

In [30]:
response("Will I get job guarantee?")

'The courses included in this bootcamp are done by 9000+ learners and many of them have secured a job which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.'

In [31]:
response("What is the duration of the course?")

'You can complete all courses in 3 months if you dedicate 2-3 hours per day.'

### Source & Reference: https://www.youtube.com/watch?v=AjQPRomyd-k&list=PLeo1K3hjS3uu0N_0W6giDXzZIcB07Ng_F&index=4&ab_channel=codebasics 